In [ ]:
!pip install ../

In [ ]:
import codex.codex as codex

# requirements
import sys
import os
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [3]:
def load_json(fp):
    with open(fp) as f:
        j = json.load(f)
    return j


def cc_results_t(set_name, coverage: dict, element: str, strengths):
    print(coverage.keys())
    for t in strengths:
        print(
            "{} | {} for {}-way interactions: {}".format(
                set_name, element, t, coverage[t][element]
            )
        )
        if t == 3:
            print()

    return  # [coverage[t][element] for t in strengths]


def sdcc_results_t(set_name, coverage: dict, element: str, strengths, direction):
    print(coverage.keys())
    for t in strengths:
        print(
            "{} {} over {}-way interactions: {}".format(
                set_name, element, t, coverage[t][direction]["SDCC"]
            )
        )
        if t == 3:
            print()

    return  # [coverage[t][direction]['SDCC'] for t in strengths]


def display_input_file(filepath):
    with open(filepath) as f:
        input_template = json.load(f)
    input_template = codex.output.output_json_readable(input_template, print_json=True)


def describe_tabular_dataset(filepath):
    abstract_df = pd.read_csv(filepath).drop("Unnamed: 0", axis=1)
    display(abstract_df)
    display(abstract_df.describe())
    for col_name in abstract_df.columns.tolist():
        display(col_name, abstract_df[col_name].describe())


def display_binning_file(filepath):
    with open(filepath) as f:
        print(f.read())


def display_split_file(filepath):
    splits_ab = load_json(filepath)
    display(list(splits_ab.keys()))
    display(codex.output.output_json_readable(splits_ab, truncate_lists=True))


def model_performance(
    train_name,
    test_name,
    train_input,
    test_input,
    train_coverage,
    test_coverage,
    metric="precision",
):
    print("Obtaining model evaluations for a model trained on {}...".format(train_name))
    perf = load_json(
        os.path.join("rareplanes_demo/performance", train_input["performance_file"])
    )
    print(
        "Performance of model trained on {} (CC_2: {}) and evaluated on {} (CC_2: {}):".format(
            train_name,
            round(train_coverage[2]["CC"], 3),
            test_name,
            round(test_coverage[2]["CC"], 3),
        )
    )
    codex.output.output_json_readable(
        perf["test"]["Overall Performance"], print_json=True
    )

    return perf["test"]["Overall Performance"][metric]


def model_performance_sdcc(train_name, test_name, train_input, metric="precision"):
    direction = test_name + "-" + train_name
    print(
        "Obtaining model evaluations for a model trained on {} and evaluated on {}...".format(
            train_name, test_name
        )
    )
    perf = load_json(
        os.path.join("rareplanes_demo", "performance", train_input["performance_file"])
    )

    codex.output.output_json_readable(
        perf["test"]["Overall Performance"], print_json=True
    )

    return perf["test"]["Overall Performance"][metric]


def visualize_cc(codex_input):
    output_path, strengths = codex.input_handler.define_experiment_variables(
        codex_input
    )
    fig, ax = plt.subplots(2, 3, figsize=(16, 12))
    fig.subplots_adjust(wspace=0.15, hspace=-0.5)

    for i in range(len(ax)):
        for j in range(len(ax[0])):
            ax[i][j].axis("off")

    for t in strengths:
        img_t = mpimg.imread(
            os.path.join(
                output_path, "CC", "CC_binary-t{}_RarePlanes_all.png".format(t)
            )
        )
        img_t_op = mpimg.imread(
            os.path.join(
                output_path,
                "CC",
                "CC_frequency_proportion_standardized-t{}_RarePlanes_all.png".format(t),
            )
        )
        ax[0][t - 1].imshow(img_t)
        ax[1][t - 1].imshow(img_t_op)
        """ax[0].axis('off')
        ax[1].axis('off')"""


def visualize_sdcc(codex_input):
    output_path, strengths = codex.input_handler.define_experiment_variables(
        codex_input
    )
    fig, ax = plt.subplots(2, 3, figsize=(20, 16))
    fig.subplots_adjust(wspace=0.15, hspace=-0.5)

    target_name = codex_input["target_name"]
    source_name = codex_input["source_name"]

    for i in range(len(ax)):
        for j in range(len(ax[0])):
            ax[i][j].axis("off")

    for t in strengths:
        img_t = mpimg.imread(
            os.path.join(
                output_path,
                "SDCC",
                "SDCC-t{}-way Set Diff {} not appearing in {}_RarePlanes_wneither.png".format(
                    t, target_name, source_name
                ),
            )
        )
        img_t_op = mpimg.imread(
            os.path.join(
                output_path,
                "SDCC",
                "SDCC-t{}-way Set Diff {} not appearing in {}_RarePlanes_wneither.png".format(
                    t, source_name, target_name
                ),
            )
        )
        ax[0][t - 1].imshow(img_t)
        ax[1][t - 1].imshow(img_t_op)
        """ax[0].axis('off')
        ax[1].axis('off')"""


def split_intersection(list1, list2):
    intersect = list(set(list1) & set(list2))
    intersect = [entry for entry in intersect if entry != "..."]

    print("The intersection of the splits contain {} entries.".format(len(intersect)))
    return intersect


def differential_performance_cc(
    train_cc_1, train_cc_2, perf_metric1, perf_metric2, metric, t
):
    plt.figure(figsize=(9, 6))
    plt.title("Combinatorial coverage against model performance")
    plt.xlim(0, 1)
    plt.ylim(0.5, 1)
    plt.xlabel("Combinatorial coverage of training set for 2-way interactions")
    plt.ylabel("Model performance evaluating on test12: {}".format(metric))
    plt.grid(visible="on")

    plt.scatter(x=train_cc_1[t]["CC"], y=perf_metric1, color="red")
    plt.scatter(x=train_cc_2[t]["CC"], y=perf_metric2, color="green")

    plt.legend(["train1", "train2"])


def differential_performance_sdcc(
    train_cc_in,
    train_cc_out,
    direction_in,
    direction_out,
    perf_metric_in,
    perf_metric_out,
    metric,
    t,
):
    plt.figure(figsize=(9, 6))
    plt.title("SDCC against model performance")
    plt.xlim(0, 1)
    plt.ylim(0.0, 1)
    plt.xlabel("SDCC of training set for 2-way interactions: test-train")
    plt.ylabel("Model performance evaluating on test: {}".format(metric))
    plt.grid(visible="on")

    plt.scatter(x=train_cc_in[t][direction_in]["SDCC"], y=perf_metric_in, color="red")
    plt.scatter(
        x=train_cc_out[t][direction_out]["SDCC"], y=perf_metric_out, color="green"
    )

    plt.legend(["train1", "train2"])


def display_sie_train_set(input_sie, feature, value):
    #'Hour_of_Day', '[10.26,11.27)'

    output_path, strengths = codex.input_handler.define_experiment_variables(input_sie)
    split_by_csv_dir = os.path.join(output_path, "splits_by_csv")

    covered_hour_0_train = pd.read_csv(
        os.path.join(split_by_csv_dir, "train_0_0_.csv")
    ).drop("Unnamed: 0", axis=1)

    display("TRAINING SET EXCLUDING Hour_of_Day: [10.26,11.27)")
    display(covered_hour_0_train.head())

    display("TRAINING SET Hour_of_Day: [10.26,11.27) SAMPLES:")
    display(
        covered_hour_0_train[
            covered_hour_0_train["Hour_of_Day"] == "[10.26,11.27)"
        ].head()
    )

    display("No samples with 'Hour_of_Day', '[10.26,11.27)'.")


def display_sie_test_sets(input_sie, feature, value):
    output_path, strengths = codex.input_handler.define_experiment_variables(input_sie)
    split_by_csv_dir = os.path.join(output_path, "splits_by_csv")

    covered_hour_0_test_excl = pd.read_csv(
        os.path.join(split_by_csv_dir, "notcovered_0_0_.csv")
    ).drop("Unnamed: 0", axis=1)
    covered_hour_0_test_incl = pd.read_csv(
        os.path.join(split_by_csv_dir, "covered_0_0_.csv")
    ).drop("Unnamed: 0", axis=1)

    display("TEST COVERED BY TRAINING SET (EXCLUDING Hour_of_Day: [10.26,11.27)):")
    display(covered_hour_0_test_incl.head())

    display(
        "TEST NOT COVERED BY TRAINING SET (ALL SAMPLES' Hour_of_Day: [10.26,11.27)):"
    )
    display(covered_hour_0_test_excl.head())


def display_sie_results(input_sie, metric):
    performance_path = os.path.join(
        input_sie["codex_directory"], input_sie["performance_folder"]
    )

    fig, ax = plt.subplots(3, 1, figsize=(20, 16))
    for i in range(len(ax)):
        ax[i].axis("off")
    img = mpimg.imread(os.path.join(performance_path, "output_{}.png".format(metric)))
    img1 = mpimg.imread(os.path.join(performance_path, "output_spread.png"))
    img2 = mpimg.imread(os.path.join(performance_path, "output_incl_training.png"))
    ax[0].imshow(img)
    ax[1].imshow(img1)
    ax[2].imshow(img2)

    return

# 0) CODEX directory
To run its experiments, CODEX utilizes a CODEX directory in which datasets, binning files, split files, performance files, experiment folders, and input files which specify these elements, reside.

CODEX can get a user started with a setup function, which can include template files for how each requirement should be formatted.

In [4]:
codex_directory_name = input("Name of CODEX directory:")
codex_directory = codex.setup_new_codex_env(
    codex_directory_name, include_templates=True
)

#### Input file)

In [5]:
# display_input_file(os.path.join(codex_directory, 'configs', 'input_TEMPLATE.json'))

#### Dataset)

In [6]:
# Tabular dataset
# describe_tabular_dataset(os.path.join(codex_directory, 'data', 'dataset_sample_abstract.csv'))

#### Binning file)

In [7]:
# display_binning_file(os.path.join(codex_directory, 'binning', 'bins_sample_abstract.txt'))

# 1) Dataset Evaluation
CODEX's most basic, mode, "dataset evaluation," computes combinatorial coverage for each specified $t$.

*"How complete is my dataset for a defined universe?"*

#### RarePlanes Dataset)

RarePlanes is an open source dataset by CosmiqWorks consisting of real and synthetic satellite images. In the real portion of the dataset, 253 distributed into 8,525 image tiles.

Observe one such tile:

The dataset also contains an associated metadata table, containing coordinates, weather, season, etc., for each tile.

In [ ]:
# RarePlanes: Sample and its metadata
from PIL import Image

img_98_104001000F15D300_tile_177 = Image.open(
    os.path.join("../resources/98_104001000F15D300_tile_177.png")
)
rareplanes_df = pd.read_csv(
    os.path.join(
        "rareplanes_demo",
        "metadata",
        "RarePlanes_Metadata_Augmented_Processed_localized-tiled-controlpt-d01-full_resampled.csv",
    )
)
rareplanes_df.index = rareplanes_df.image_tile_id
display(img_98_104001000F15D300_tile_177)
display(rareplanes_df.loc["98_104001000F15D300_tile_177"])

### 1a) Combinatorial coverage over a defined universe



In [ ]:
# Dataset
rareplanes_df.head(n=5)

In [ ]:
# Binning file, binned dataset
display_binning_file(os.path.join("rareplanes_demo", "binning", "bins-signif.txt"))

In [ ]:
input_cc_train1 = load_json(
    os.path.join("rareplanes_demo", "configs", "cc_train1.json")
)
universe, rareplanes_df_binned = codex.universe_handler.define_input_space(
    input_cc_train1
)

# rareplanes_df_binned = pd.read_csv(os.path.join('rareplanes_demo', 'metadata', 'RarePlanes_Metadata_Augmented_Processed_localized-tiled-controlpt-d01-full_resampled_Binned.csv'))
display(rareplanes_df_binned.head())
display(universe)

In [ ]:
# Input file
input_cc_train1 = load_json(
    os.path.join("rareplanes_demo", "configs", "cc_train1.json")
)
display(codex.output.output_json_readable(input_cc_train1))

### 1b) Run dataset evaluation on different datasets
From the overall dataset:
- Withhold a randomly sampled test set from the RarePlanes overall dataset first
    - Size: 215 samples
- Construct a training set with artificially low coverage, train1
    - By including only samples captured in "Temperate Grasslands, Savannas & Shrublands" biomes
    - Size: 1,217 samples
- Construct a training set with moderate coverage by randomly sampling from remaining samples, train2
    - Size: 1,217 samples

In [ ]:
# Run CODEX
train1_coverage = codex.run(input_cc_train1, verbose="1")

In [ ]:
# JSON results: combinatorial coverage: __demo__/rareplanes_demo/demo_1-coverage_train1/coverage.json
display(train1_coverage)
train1_cc = cc_results_t("Train 1", train1_coverage, "CC", [1, 2, 3])

In [ ]:
# JSON results: missing interactions: __demo__/rareplanes_demo/demo_1-coverage_train1/coverage.json
# TRY other result contents: ['count appearing interactions', 'total possible interactions', 'combinations', 'combination counts']
element = "missing interactions"
cc_results_t("Train 1", coverage=train1_coverage, element=element, strengths=[1, 2, 3])

In [ ]:
# Visualization results: __demo__/rareplanes_demo/demo_1-coverage_train1/CC/
visualize_cc(input_cc_train1)

In [ ]:
# Train 2: Load, run, results
input_cc_train2 = load_json(
    os.path.join("rareplanes_demo", "configs", "cc_train2.json")
)
display(input_cc_train2)
# Note that the config_id and dataset fields have changed

In [ ]:
train2_coverage = codex.run(input_cc_train2, verbose="1")
cc_results_t("Train 2", train2_coverage, "CC", [1, 2, 3])
visualize_cc(input_cc_train2)

In [ ]:
# Test12: Load, run, results
input_cc_test12 = load_json(
    os.path.join("rareplanes_demo", "configs", "cc_test12.json")
)
test12_coverage = codex.run(input_cc_test12, verbose="1")
cc_results_t("Test12", test12_coverage, "CC", [1, 2, 3])
visualize_cc(input_cc_test12)

- Dataset evaluation allows the user to test **how completely the dataset covers that input space.**

### 1c) Model performance and dataset evaluation
Furthermore, depending on how the universe is defined, combinatorial coverage over the universe resulting from dataset evaluation can be correlated with model performance. If a universe describes the operating envelope poorly, CC is misleading.

In [ ]:
# CC of training set vs. model performance
# SELECT metric: [precision, recall, f1]
metric = "f1"
performance_train1 = model_performance(
    "train1",
    "test12",
    input_cc_train1,
    input_cc_test12,
    train1_coverage,
    test12_coverage,
    metric=metric,
)
performance_train2 = model_performance(
    "train2",
    "test12",
    input_cc_train2,
    input_cc_test12,
    train2_coverage,
    test12_coverage,
    metric=metric,
)

differential_performance_cc(
    train1_coverage, train2_coverage, performance_train1, performance_train2, metric, 2
)

From this example with this particular defined universe, higher coverage over the universe is correlated with higher model performance. Depending on the formulation of the particular operating envelope, coverage can be a helpful metric of performance but also a helpful predictor of performance.

This example demonstrates the capability of dataset evaluation. Dataset evaluation enables a user to characterize how complete a dataset is over a defined universe. In this case, the test set, as is the train2 set, is relatively complete over the defined universe. However, there likely exist scenarios in which the model operates in an environment that is unlike the complete universe. Because of this, there is a need to characterize the relationship between two sets of data that may have different envelopes, as can be the case when datasets are split into training and test sets.

### 1d) Try: Bin adjustment

Navigating to 'rareplanes_demo/binning/bins-signif.txt', try re-adjusting a bins of continuous variables to reconstruct the universe and reeavluate the dataset.

**Note: Bins should cover all values between the minimum and the maximum of each feature.** Run the following cells afterward.

In [ ]:
display_binning_file(os.path.join("rareplanes_demo", "binning", "bins-edit.txt"))

In [ ]:
input_cc_train1 = load_json(
    os.path.join("rareplanes_demo", "configs", "cc_train1.json")
)
input_cc_train1["bin_file"] = "binning/bins-edit.txt"
results_cc_train1 = codex.run(input_cc_train1, verbose="1")
cc_results_t("Train 1, bins readjusted:", results_cc_train1, "CC", [1, 2, 3])
visualize_cc(input_cc_train1)

# 2) Dataset Split Evaluation
"Dataset split evaluation" computes set difference combinatorial coverage (SDCC) between training, validation, and test datasets.

*"How do I characterize the difference and distance between training/validation/testing splits of my dataset?"*

### Differing operating envelopes
For simplicity, consider two completely separate regions, A and B, of the RarePlanes dataset. These regions might exist as the different operating envelopes environments a model is deployed in:
- Region A was constructed by selecting samples containing the three 2-way interactions of level 1 of the Hour_of_Day metadata feature and levels 1, 2, and 3 of the avg_pan_resolution metadata feature.
    - Samples containing h1\*p1, h1\*p2, h1\*p3
- Region B was constructed by selecting samples containing the three 2-way interactions of level 2 of the Hour_of_Day metadata feature and levels 1, 2, and 3 of the avg_pan_resolution metadata feature.
    - Samples containing h2\*p1, h2\*p2, h2\*p3
- size(regionA) = size(regionB)

Because of this construction, A and B are disjoint in 2-way interactions and therefore disjoint in samples at the 3-way interaction level as well as being disjoint in their samples.

In each separate region, training and testing **splits** are constructed by random sampling.
- trainA, testA is randomly sampled 85-15.
- trainB, testB is randomly sampled 85-15.

### Dataset splits)
Dataset splits are leveraged by CODEX in the form of split files to compute CC values of splits and SDCC values between splits. 

CODEX supports split files in specific JSON format to select samples for each split.
- Lists under keys designating the sample names belonging to each split as they appear in the dataset ('train', 'validation', 'test')
- Split ID for user record

In [ ]:
display_split_file(os.path.join("rareplanes_demo", "splits", "split_AB_combined.json"))

In [ ]:
# Load and view splits of sample ID's provided in file
splits_ab = load_json(
    os.path.join("rareplanes_demo", "splits", "split_AB_combined.json")
)

trainA_ids = splits_ab["trainA"]
testA_ids = splits_ab["testA"]
trainB_ids = splits_ab["trainB"]
testB_ids = splits_ab["testB"]
print("{} samples in train A: {}".format(len(trainA_ids), trainA_ids[:2] + ["..."]))
print("{} samples in test A: {}".format(len(testA_ids), testA_ids[:2] + ["..."]))
print("")
print("{} samples in train B: {}".format(len(trainB_ids), trainB_ids[:2] + ["..."]))
print("{} samples in test B: {}".format(len(testB_ids), testB_ids[:2] + ["..."]))
print("")
intersection = split_intersection(trainA_ids, testA_ids)

### 2a) Dataset Split Evaluation: Similar Operating Envelopes

In [ ]:
# Dataset
rareplanes_df = pd.read_csv(
    os.path.join(
        "rareplanes_demo",
        "metadata",
        "RarePlanes_Metadata_Augmented_Processed_localized-tiled-controlpt-d01-full_resampled.csv",
    )
)
display(rareplanes_df.head())

In [ ]:
# Binning file, binned dataset
display_binning_file(os.path.join("rareplanes_demo", "binning", "bins-signif.txt"))

In [ ]:
# Universe and binned dataset
input_sdcc_tAeA = load_json(os.path.join("rareplanes_demo", "configs", "sdcc_A-A.json"))
universe, rareplanes_df_binned = codex.universe_handler.define_input_space(
    input_sdcc_tAeA
)
display(universe)
display(rareplanes_df_binned.head())

In [ ]:
# Load input files
input_sdcc_tAeA = load_json(os.path.join("rareplanes_demo", "configs", "sdcc_A-A.json"))
display(input_sdcc_tAeA)
input_sdcc_tBeB = load_json(os.path.join("rareplanes_demo", "configs", "sdcc_B-B.json"))
display(input_sdcc_tBeB)
# Note changes to the fields: mode, config_id, split_folder, split_file, source_tag, target_tag

In [ ]:
# SDCC within region A
results_sdcc_tAeA = codex.run(input_sdcc_tAeA, verbose="1")

# Train interactions not in test
sdcc_results_t("A: train-test", results_sdcc_tAeA, "SDCC", [1, 2, 3], "trainA-testA")
# Test interactions not in train
sdcc_results_t("A: test-train", results_sdcc_tAeA, "SDCC", [1, 2, 3], "testA-trainA")

In [ ]:
visualize_sdcc(input_sdcc_tAeA)

In [ ]:
# SDCC within region B
results_sdcc_tBeB = codex.run(input_sdcc_tBeB, verbose="1")

# Train interactions not in test
sdcc_results_t("B: train-test", results_sdcc_tBeB, "SDCC", [1, 2, 3], "trainB-testB")
# Test interactions not in train
sdcc_results_t("B: test-train", results_sdcc_tBeB, "SDCC", [1, 2, 3], "testB-trainB")

In [ ]:
visualize_sdcc(input_sdcc_tBeB)

For both region A and region B, every interaction appearing in the test set appears in the training set, and every interaction appearing in the trianing set appears in the test set.

### 2b) Dataset Split Evaluation: Differing Operating Envelopes

In [33]:
# Load input files
input_sdcc_tAeB = load_json(
    os.path.join("rareplanes_demo", "configs", "sdcc_trnA-testB.json")
)
input_sdcc_tBeA = load_json(
    os.path.join("rareplanes_demo", "configs", "sdcc_trnB-testA.json")
)

In [ ]:
# Dataset split comparison across regions
results_sdcc_tAeB = codex.run(input_sdcc_tAeB, verbose="1")

# Train interactions not in test
sdcc_results_t(
    "Train A, test B: train-test", results_sdcc_tAeB, "SDCC", [1, 2, 3], "trainA-testB"
)
# Test interactions not in train
sdcc_results_t(
    "Train A, test B: test-train", results_sdcc_tAeB, "SDCC", [1, 2, 3], "testB-trainA"
)

In [ ]:
visualize_sdcc(input_sdcc_tAeB)

In [ ]:
# Dataset split comparison across regions
results_sdcc_tBeA = codex.run(input_sdcc_tBeA, verbose="1")

# Train interactions not in test
sdcc_results_t(
    "Train B, test A: train-test", results_sdcc_tBeA, "SDCC", [1, 2, 3], "trainB-testA"
)
# Test interactions not in train
sdcc_results_t(
    "Train B, test A: test-train", results_sdcc_tBeA, "SDCC", [1, 2, 3], "testA-trainB"
)

In [ ]:
visualize_sdcc(input_sdcc_tBeA)

### 2c) Dataset split evaluation and model performance
Assuming or knowing that the model performs well over the entire defined universe, SDCC that results from dataset split evaluation can be a predictor of how a model will perform given its difference from the operating envelope of the training set.

In [ ]:
# SDCC of training set vs. model performance
# SELECT metric: [precision, recall, f1]
metric = "f1"
performance_trainAA = model_performance_sdcc(
    "testA", "trainA", input_sdcc_tAeA, metric=metric
)
performance_trainAB = model_performance_sdcc(
    "testB", "trainA", input_sdcc_tAeB, metric=metric
)

In [ ]:
metric = "precision"  # [precision, f1, recall]
differential_performance_sdcc(
    results_sdcc_tAeA,
    results_sdcc_tAeB,
    "testA-trainA",
    "testB-trainA",
    performance_trainAA,
    performance_trainAB,
    metric,
    2,
)

When deployed outside of its operating envelope, model performance degrades.

# 3) Systematic Inclusion and Exclusion (SIE)

The SIE experiment aims to discover important combinations of features by systematically withholding each interaction from training and comparing a model's performance on one test set also withholding the interaction and one that contains only samples with said interaction.

*Beyond a feature-value level, what combinations of features and their interactions are most critical for the model to see in the dataset during training?*

In [ ]:
# Dataset
rareplanes_df = pd.read_csv(
    os.path.join(
        "rareplanes_demo",
        "metadata",
        "RarePlanes_Metadata_Augmented_Processed_localized-tiled-controlpt-d01-full_resampled.csv",
    )
)
display(rareplanes_df.head())

In [ ]:
# Binning file, binned dataset
display_binning_file(os.path.join("rareplanes_demo", "binning", "bins-complete.txt"))

In [ ]:
# Universe and binned dataset
input_sie = load_json(os.path.join("rareplanes_demo", "configs", "sie.json"))
universe, rareplanes_df_binned = codex.universe_handler.define_input_space(input_sie)
display(rareplanes_df_binned.head())
display(universe)

In [ ]:
input_sie = load_json(os.path.join("rareplanes_demo", "configs", "sie.json"))
results = codex.run(input_sie, verbose="1")

print("Results contents:", list(results.keys()))

In [ ]:
display_sie_train_set(input_sie, "Hour_of_Day", "[10.26,11.27)")

In [ ]:
display_sie_test_sets(input_sie, "Hour_of_Day", "[10.26,11.27)")

In [ ]:
# Visualizations: SIE included/excluded training
metric = "precision"  # [precision, recall, f1]
display_sie_results(input_sie, metric=metric)